In [1]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', None)

In [2]:
from helper import GenomeCreator

gc = GenomeCreator()
path_to_daily_csv = "~/class/f24/cbm/evo-life/prepped/daily.csv"
gc.load_data_from_csv()
tmp_genome = gc.create_genome_for_individual(id="621e32e667b776a2406d2f1c")

In [39]:
from pprint import pprint
daily = pd.read_csv("~/class/f24/cbm/evo-life/prepped/daily.csv")
individuals = pd.read_csv("~/class/f24/cbm/evo-life/prepped/individuals.csv")
pprint(daily.columns)

Index(['Unnamed: 0', 'id', 'date', 'nremhr', 'rmssd', 'spo2', 'stress_score',
       'sleep_points_percentage', 'exertion_points_percentage',
       'responsiveness_points_percentage', 'distance', 'activityType', 'bpm',
       'lightly_active_minutes', 'moderately_active_minutes',
       'very_active_minutes', 'sedentary_minutes', 'mindfulness_session',
       'sleep_duration', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency',
       'gender', 'bmi', 'TENSE/ANXIOUS', 'TIRED', 'GYM', 'HOME', 'OUTDOORS',
       'day'],
      dtype='object')


In [4]:
from models import Genome
genome = Genome(None, None)
phenotype_data = gc.get_all_phenotype_data()
phenotypes = []
for k, v in phenotype_data.items():
    phenotypes.append(genome.calculate_genome_phenotype(v['anxiety'], v['tired'], v['stress_score'], v['sleep_points_percentage']))

phenotypes

[np.float64(0.4891969936708861),
 np.float64(0.724451754385965),
 np.float64(0.6489489992103546),
 np.float64(0.7594551976503443),
 np.float64(0.7138407367280606),
 np.float64(0.6048701298701299),
 np.float64(0.6385690789473684),
 np.float64(0.6830122180451128),
 np.float64(0.5638235294117646),
 np.float64(0.7484126984126984),
 np.float64(0.816184101286841),
 np.float64(0.7419764157604859),
 np.float64(0.7481645606645607),
 np.float64(0.6804704822616408),
 np.float64(0.6567441932295528),
 np.float64(0.7924237508111616)]

In [3]:
daily = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/csv_rais_anonymized/daily_fitbit_sema_df_unprocessed.csv")
hourly = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/csv_rais_anonymized/hourly_fitbit_sema_df_unprocessed.csv")
breq = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/breq.csv")
panas = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/panas.csv")
personality = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/personality.csv")
stai = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/stai.csv")
ttm = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/scored_surveys/ttm.csv")

/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_45334/255447001.py:2: DtypeWarning: Columns (5,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  hourly = pd.read_csv("~/class/f24/cbm/evo-life/data/rais_anonymized/csv_rais_anonymized/hourly_fitbit_sema_df_unprocessed.csv")


In [4]:
# preprocessing the daily data
daily.drop("Unnamed: 0", axis=1)

# all values will be associated with the ID, let's count how many there are for each
num_ids = daily['id'].nunique() # 71

# is there an even distribution of dates for the number of records we have (does each ID have the same set of dates?)
# subset = daily.loc[daily['id'].isin(["621e2e8e67b776a24055b564"])]
subset = daily.groupby("id").nunique()
subset = subset.reset_index()
date_per_id_counts = subset[['id', 'date']]

In [ ]:
nan_values = daily.isna()
nan_values = nan_values.reset_index()
true_nan_values = nan_values.sum()
true_nan_values

In [ ]:
ids = list(hourly['id'].unique())

In [ ]:
hourly = hourly.drop('Unnamed: 0', axis=1)

In [ ]:
all_cols = ["id", "date", "hour","distance", "steps", "calories", "age", "bpm", "bmi", "gender",
             "mindfulness_session", "SAD", "TIRED", "TENSE/ANXIOUS", "ENTERTAINMENT", "GYM", "HOME",
             "OUTDOORS"]
numeric_coerce = ["distance", "steps", "calories","mindfulness_session", "SAD", "TIRED", "TENSE/ANXIOUS", "ENTERTAINMENT", "GYM", "HOME",
             "OUTDOORS"]
tmp = hourly[all_cols]
tmp[numeric_coerce] = tmp[numeric_coerce].apply(pd.to_numeric, errors="coerce").fillna(0).astype(np.int64)
# first 35 ids are good, fails on 36 and after that for some reason

# test_id_subset = ids[36]
# tmp = tmp.loc[hourly['id'] == test_id_subset]

In [1]:
roll_up = tmp.groupby(["id", "date"]).agg({
    "distance": ['sum'],
    "steps": ['sum'],
    "calories": ['sum'],
    "gender": ['first'],
    'age': ['first'],
    # 'bmi': ['max'],
    "mindfulness_session": ['max'],
    "SAD": ['max'],
    "TIRED": ["max"],
    "TENSE/ANXIOUS": ['max'],
    "ENTERTAINMENT": ['max'],
    "GYM": ["max"],
    "HOME": ['max'],
    "OUTDOORS": ['max']
    })
roll_up.reset_index()
roll_up


NameError: name 'tmp' is not defined

In [ ]:
tmp['age'].unique()

In [5]:
daily = daily.drop('Unnamed: 0', axis=1)

In [6]:
# need to join the aggregated hourly data to the daily data here to get a full dataset for us to use. 
to_use = ['id', 'date', 'nremhr', 'rmssd', 'spo2', 'stress_score', 'sleep_points_percentage',
          'exertion_points_percentage', 'responsiveness_points_percentage', 'distance', 'activityType',
          'bpm', 'lightly_active_minutes', 'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes',
          'mindfulness_session', 'sleep_duration', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency', 'gender',
          'bmi', 'TENSE/ANXIOUS', 'TIRED', "GYM", "HOME", "OUTDOORS"]
d = daily[to_use]

In [7]:
median_values_impute = ["nremhr", "rmssd", "spo2", "stress_score", "sleep_points_percentage", 
                        "exertion_points_percentage", "responsiveness_points_percentage", "distance", 
                        "minutesAsleep", "minutesAwake", "sleep_efficiency", "bpm", 
                        "lightly_active_minutes", "moderately_active_minutes", "very_active_minutes"]
d[median_values_impute] = d[median_values_impute].astype(np.float64)

median_values = d.groupby('id')[median_values_impute].median()

d[median_values_impute] = d[median_values_impute].fillna(median_values)
d = d.fillna(0)

/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_45334/2040797843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[median_values_impute] = d[median_values_impute].astype(np.float64)
/var/folders/_m/1l5g3ybj3tq0zbv9zsff88c00000gn/T/ipykernel_45334/2040797843.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[median_values_impute] = d[median_values_impute].fillna(median_values)


In [8]:
# convert dates to numbered days
d = d.sort_values(by="date")
d['day'] = d.groupby("id").cumcount()
d.to_csv("~/class/f24/cbm/evo-life/prepped/daily.csv")

In [6]:
pd.read_csv("~/class/f24/cbm/evo-life/prepped/daily.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Tripp/class/f24/cbm/evo-life/prepped/daily.csv'

In [8]:
pd.read_csv("../prepped/daily.csv")

,Unnamed: 0,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS,day
0,6368,621e362467b776a2404ad513,2021-04-08,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0
1,6369,621e362467b776a2404ad513,2021-04-09,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1
2,6370,621e362467b776a2404ad513,2021-04-10,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,2
3,6371,621e362467b776a2404ad513,2021-04-12,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,3
4,6372,621e362467b776a2404ad513,2021-04-13,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,3498,621e32d067b776a2405b7d54,2022-01-22,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,FEMALE,22.0,0.0,0.0,0.0,0.0,0.0,84
7406,1908,621e309267b776a240ae1cdb,2022-01-22,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,MALE,<19,0.0,0.0,0.0,0.0,0.0,76
7407,2535,621e310d67b776a24003096d,2022-01-22,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,FEMALE,20.0,0.0,0.0,0.0,0.0,0.0,109
7408,2081,621e30b267b776a240c5e13f,2022-01-22,0.000,0.00,0.0,0.0,0.0,0.000,0.000000,0.0,0,0.0,0.0,0.0,0.0,1440.0,False,0.0,0.0,0.0,0.0,FEMALE,21.0,0.0,0.0,0.0,0.0,0.0,67


In [ ]:
curr_df = gc.df
curr_df[curr_df['TENSE/ANXIOUS'] != 0.0]

,id,date,nremhr,rmssd,spo2,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,sleep_duration,minutesAsleep,minutesAwake,sleep_efficiency,gender,bmi,TENSE/ANXIOUS,TIRED,GYM,HOME,OUTDOORS,day
0,621e362467b776a2404ad513,2021-04-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0
1,621e362467b776a2404ad513,2021-04-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1
2,621e362467b776a2404ad513,2021-04-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,2
3,621e362467b776a2404ad513,2021-04-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,3
4,621e362467b776a2404ad513,2021-04-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,4


# Behavioral Survey Stuff

In [ ]:
# find out how many days each id has
days_per_id = d.groupby('id').agg(
    {
        "date": ['count']
    }
)
print("Total number of IDs", days_per_id.__len__())
days_per_id

In [ ]:
# convert dates to numbered days
d = d.sort_values(by="date")
d['day'] = d.groupby("id").cumcount()
d.loc[d['id']=="621e346f67b776a24081744f"].sort_values(by="day")


In [ ]:
number_records_per_id_breq = breq.groupby("user_id").agg(
    {
        "breq_introjected_regulation": "count"
    }
)
number_records_per_id_breq.count()

In [ ]:
breq['user_id'].count()

In [ ]:
print("Number of records in panas:", panas['user_id'].count())
number_records_per_id_panas = panas.groupby("user_id").agg(
    {
        "positive_affect_score": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_panas.count())
number_records_per_id_panas

In [ ]:
personality.columns

In [ ]:
print("Number of records in personality:", personality['user_id'].count())
number_records_per_id_personality = personality.groupby("user_id").agg(
    {
        "extraversion": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_personality.count())
number_records_per_id_personality

In [ ]:
stai.columns

In [ ]:
print("Number of records in stai:", stai['user_id'].count())
number_records_per_id_stai = stai.groupby("user_id").agg(
    {
        "stai_stress": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_stai.count())
number_records_per_id_stai

In [ ]:
ttm.columns

In [ ]:

print("Number of records in ttm:", ttm['user_id'].count())
number_records_per_id_ttm = ttm.groupby("user_id").agg(
    {
        "ttm_consciousness_raising": "count"
    }
)
print("Number of individual ids represented:", number_records_per_id_ttm.count())
number_records_per_id_ttm

In [ ]:
# let's take subsets of each of the behavioral tables and then join them together so that we have
# one table that contains behavioral information on each person who took the surveys

# problem: some people have multiple records in the behavior tables. Do I want to average those values together

# my decision is to do an average for each value

breq_averaged = breq.groupby("user_id").agg(
    {
        "breq_amotivation":"mean",
        "breq_external_regulation": "mean", 
        "breq_introjected_regulation": "mean", 
        "breq_identified_regulation": "mean", 
        "breq_intrinsic_regulation": "mean", 
        "breq_self_determination": "first"
    }
)
breq_averaged = breq_averaged.reset_index()
print("BREQ ID COUNT:", breq_averaged['user_id'].count())

panas_averaged = panas.groupby("user_id").agg(
    {
        "positive_affect_score":"mean",
        "negative_affect_score": "mean"
    }
)
panas_averaged = panas_averaged.reset_index()
print("PANAS ID COUNT:", panas_averaged['user_id'].count())

personality_averaged = personality.groupby("user_id").agg(
    {
        "agreeableness": "mean",
        "conscientiousness": "mean",
        "stability": "mean", 
        "intellect": "mean"
    }
)

personality_averaged = personality_averaged.reset_index()
print("PERSONALITY ID COUNT:", personality_averaged['user_id'].count())

stai_averaged = stai.groupby("user_id").agg(
    {
        "stai_stress": "mean",
        "stai_stress_category" : "first"
    }
)

stai_averaged = stai_averaged.reset_index()
print("STAI ID COUNT:", stai_averaged['user_id'].count())

ttm_averaged = ttm.groupby("user_id").agg(
    {
        "ttm_consciousness_raising": "mean", 
        "ttm_dramatic_relief": "mean", 
        "ttm_environmental_reevaluation": "mean", 
        "ttm_self_reevaluation": "mean", 
        "ttm_social_liberation": "mean", 
        "ttm_counterconditioning": "mean", 
        "ttm_helping_relationships": "mean",
        "ttm_reinforcement_management": "mean",
        "ttm_self_liberation": "mean", 
        "ttm_stimulus_control": "mean"
    }
)

ttm_averaged = ttm_averaged.reset_index()
print("TTM ID COUNT:", ttm_averaged['user_id'].count())




In [ ]:
step_one = breq_averaged.merge(panas_averaged, on='user_id', how='outer')
step_two = step_one.merge(personality_averaged, on='user_id', how='outer')
step_three = step_two.merge(stai_averaged, on='user_id', how='outer')
individuals = step_three.merge(ttm_averaged, on='user_id', how='outer')
individuals

In [ ]:
individuals[individuals.select_dtypes(include=['float']).columns] = individuals.select_dtypes(include=['float']).fillna(0)
individuals[individuals.select_dtypes(include=['object']).columns] = individuals.select_dtypes(include=['object']).fillna('undefined')
individuals

In [ ]:
individuals.to_csv("~/class/f24/cbm/evo-life/data/prepped/individuals.csv")

In [ ]:
d.sort_values(by=['id', 'date'])

In [ ]:
d.head(100)

In [ ]:
# going to need to filter down the dataframe to the participants that actually had good data
tmp = daily.loc[daily['nightly_temperature'] > 0 ]
tmp['id'].nunique()

In [ ]:
d.apply(lambda row: (row == 0).sum(), axis=1)

In [ ]:
d.tail(100)

In [ ]:
from helper import GenomeCreator

gc = GenomeCreator()
path_to_daily_csv = "~/class/f24/cbm/evo-life/prepped/daily.csv"
gc.load_data_from_csv(path = path_to_daily_csv)



In [1]:
gc.create_genome_for_individual(id="621e32e667b776a2406d2f1c")

NameError: name 'gc' is not defined

In [ ]:
final_string = ""
i = 0
for c in list(d.columns):
    # if i % 5 == 0:
    #     final_string += '\n'
    # i += 1
    final_string += c + " "
final_string